### CacheBackedEmbeddings

대용량의 문서를 임베딩할떄는 금액이 많이 나온다. 재계산을 피하기위해 저장되거나, 일시적으로 캐시할 수 있다.

`CacheBackedEmbeddings`

1. 임베딩을하고, 파일로 저장한다..
2. 임베딩을 할때, 파일안에 캐싱되어있는 임베딩이 있는지 물어본다. 이게 있으면 바로 사용한다.


##### `LocalFileStore`에서 임베딩 사용 - 영구적 사용방법

In [1]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings

embedding = OpenAIEmbeddings(model='text-embedding-3-small')

# local file store 설정
store = LocalFileStore("./cache/")

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedding, #모델
    document_embedding_cache=store, # 어디 파일에 저장할지 정함.
    namespace=embedding.model # 지정을 안해도 되지만, 구분자를 주기위해 사용. /기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩 생성.
)

# cached_embedder = CacheBackedEmbeddings.from_bytes_store(
#     embedding, store
# )
# 코드로 해도 되더라!

In [2]:
list(store.yield_keys())

['text-embedding-3-small0f7bb237-10d9-5cca-ad15-328f0486ce30',
 'text-embedding-3-small1c33a092-eab7-5600-8613-b3a31c281857',
 'text-embedding-3-small41e7391b-b68f-5e9f-bb07-3609bb83c3e2',
 'text-embedding-3-small4f3fc01f-5e6d-537d-8c9a-284e532c7487',
 'text-embedding-3-small742b1aa5-fbc1-5ec2-a680-99cc5a4460e3',
 'text-embedding-3-smallc3acda7d-1a91-5422-ac24-2f0fb86de905',
 'text-embedding-3-smallf267cc06-16cc-540c-92bf-2c9e53c93388']

In [3]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.vectorstores import FAISS

raw_documents = TextLoader("./data/appendix-keywords.txt", encoding='utf-8').load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

documents = text_splitter.split_documents(raw_documents)

In [4]:
%time db = FAISS.from_documents(documents, embedding)

CPU times: total: 500 ms
Wall time: 1.8 s


In [5]:
# 캐싱된 임베딩을 사용하여 FAISS 데이터베이스 생성.
%time db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: total: 15.6 ms
Wall time: 12 ms


##### `InmemoryByteStore` 사용 - 비영구적 사용방법

사용자가 떠나면 모두 날라간다.

In [6]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore
store = InMemoryByteStore()

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embedding, store, namespace=embedding.model
)